In [1]:
# !pip install openmeteo-requests
# !pip install requests-cache retry-requests numpy pandas
# https://open-meteo.com/en/docs/historical-weather-api


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for requests-cache from https://files.pythonhosted.org/packages/cc/23/863f517b5d74e297e669c88093ecd92e13a06c488e90acbd5507819049ac/requests_cache-1.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for retry-requests from https://files.pythonhosted.org/packages/b1/f3/8ce908497bebbc2790ef06240a2c0fb28c096abb59062d88f85090464a5f/retry_requests-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for cattrs>=22.2 from https://files.pythonhosted.org/packages/b3/0d/cd4a4071c7f38385dc5ba91286723b4d1090b87815db48216212c6c6c30e/cattrs-23.2.3-py3-none-any.whl.metadata
  Obtaining dependency information for url-normalize>=1.4 from https://files.pythonhosted.org/packages/65/1c/6c6f408be78692fc850006a2b6dea37c2b8592892534e09996e401efc74b/url_normalize-1.4.3-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/61.4 kB ? eta -:--:--
   ---------------------------------------- 61.4/61.4 kB 1.6 MB/s eta 0:00:00
  


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.50196,
	"longitude": 17.21493,
	"start_date": "1960-05-12",
	"end_date": "1960-05-26",
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_speed_100m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 40.52724075317383°N 17.1610164642334°E
Elevation 17.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                         date  temperature_2m  wind_speed_10m  wind_speed_100m
0   1960-05-12 00:00:00+00:00       13.991500        5.241679         7.491114
1   1960-05-12 01:00:00+00:00       14.341499        5.760000         7.754637
2   1960-05-12 02:00:00+00:00       14.491500        5.771239         7.862518
3   1960-05-12 03:00:00+00:00       14.141500        5.447788         7.517021
4   1960-05-12 04:00:00+00:00       12.691500        5.154416         6.489992
..                        ...             ...             ...              ...
355 1960-05-26 19:00:00+00:00       18.391500       26.932714        37.585098
356 1960-05-26 20:00:00+00:00       17.841499       26.319420        37.446922
357 1960-05-26 21:00:00+00:00       17.391500       27.002399        38.813274
358 1960-05-26 22:00:00+00:00       17.041500       25.984919        36.656414
359 196